In [16]:
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta

def generate_youtube_logs_custom(num_rows=20000):
    random.seed(42)
    user_ids = [f"user_{random.randint(1000, 9999)}" for _ in range(500)]
    video_ids = [f"video_{random.randint(100000, 999999)}" for _ in range(300)] 
    categories = ["Music", "Gaming", "Education", "Comedy", "News", "Sports", "Travel", "Lifestyle"] 
    devices = ["Web", "Android", "iOS", "Smart TV"] 
    start_date = datetime(2022, 1, 1)  
    
    data = []
    for _ in range(num_rows):
        user_id = random.choice(user_ids)
        video_id = random.choice(video_ids) 
        category = random.choice(categories) 
        device = np.random.choice(devices, p=[0.4, 0.35, 0.2, 0.05])
        timestamp = start_date + timedelta(seconds=random.randint(0, 31536000)) 
        watch_duration = np.random.exponential(scale=20) 
        data.append([user_id, timestamp.strftime("%Y-%m-%d %H:%M:%S"), video_id, category, device, watch_duration])
    
    columns = ["user_id", "timestamp", "video_id", "category", "device", "watch_duration"]
    df = pd.DataFrame(data, columns=columns)
    return df

youtube_logs_custom = generate_youtube_logs_custom(20000)

youtube_logs_custom.to_csv("youtube_logs_custom.csv", index=False)



In [17]:
# Добавление дискретного показателя
bins = [-np.inf, 5, 20, 60, np.inf]
labels = ["very_short", "short", "medium", "long"]
youtube_logs_custom['watch_duration_category'] = pd.cut(youtube_logs_custom['watch_duration'], bins=bins, labels=labels)

In [18]:
youtube_logs_custom.head()

,user_id,timestamp,video_id,category,device,watch_duration,watch_duration_category
0,user_4946,2022-04-26 01:13:01,video_609232,Comedy,Web,13.390034,short
1,user_2827,2022-04-28 09:20:17,video_968962,Education,Web,18.343600,short
2,user_9935,2022-04-17 12:46:00,video_468900,Education,Web,29.448463,medium
3,user_1106,2022-09-28 20:24:04,video_998348,News,Android,16.954173,short
4,user_8305,2022-02-09 20:11:10,video_752483,Lifestyle,iOS,13.873868,short


In [19]:
segment_web = youtube_logs_custom[youtube_logs_custom['device'] == 'Web']
segment_ios = youtube_logs_custom[youtube_logs_custom['device'] == 'iOS']
segment_news = youtube_logs_custom[youtube_logs_custom['category'] == 'News']
segment_music = youtube_logs_custom[youtube_logs_custom['category'] == 'Music']

results = {}

In [20]:
segment_ios.head()

,user_id,timestamp,video_id,category,device,watch_duration,watch_duration_category
4,user_8305,2022-02-09 20:11:10,video_752483,Lifestyle,iOS,13.873868,short
16,user_7304,2022-09-14 11:38:11,video_132537,News,iOS,36.299768,medium
17,user_3045,2022-03-09 12:03:34,video_497519,Comedy,iOS,10.598586,short
20,user_1434,2022-10-02 13:18:21,video_801978,News,iOS,6.814110,short
22,user_3591,2022-06-15 02:34:56,video_271720,Comedy,iOS,6.450724,short


Гипотеза 1 (о медианах для непрерывного показытеля): Показатель: watch_duration (длительность просмотра)

Нулевая гипотеза (H₀): Медианы длительности просмотра для категорий устройств Web и Android равны.

Альтернативная гипотеза (H₁): Медианы длительности просмотра для категорий устройств Web и Android различаются.

In [22]:
from scipy.stats import mannwhitneyu, ks_2samp, chi2_contingency
web_watch_duration = youtube_logs_custom.loc[youtube_logs_custom['device'] == 'Web', 'watch_duration']
android_watch_duration = youtube_logs_custom.loc[youtube_logs_custom['device'] == 'Android', 'watch_duration']
stat1, p1 = mannwhitneyu(web_watch_duration, android_watch_duration, alternative='two-sided')


Гипотеза 2 (о распределении для непрерывного показытеля): Показатель: watch_duration (длительность просмотра)

Нулевая гипотеза (H₀): Распределение длительности просмотра для категорий контента Music и Gaming одинаково.

Альтернативная гипотеза (H₁): Распределение длительности просмотра для категорий контента Music и Gaming различается.

In [23]:
music_watch_duration = youtube_logs_custom.loc[youtube_logs_custom['category'] == 'Music', 'watch_duration']
gaming_watch_duration = youtube_logs_custom.loc[youtube_logs_custom['category'] == 'Gaming', 'watch_duration']
stat2, p2 = ks_2samp(music_watch_duration, gaming_watch_duration)

Гипотеза 3 (о медиане для дискретного показытеля): Показатель: watch_duration_category (категория длительности просмотра)

Нулевая гипотеза (H₀): Медианные категории длительности просмотра для устройств iOS и Smart TV равны.

Альтернативная гипотеза (H₁): Медианные категории длительности просмотра для устройств iOS и Smart TV различаются.

In [24]:
ios_watch_category = youtube_logs_custom.loc[youtube_logs_custom['device'] == 'iOS', 'watch_duration_category'].value_counts().sort_index()
smart_tv_watch_category = youtube_logs_custom.loc[youtube_logs_custom['device'] == 'Smart TV', 'watch_duration_category'].value_counts().sort_index()
stat3, p3 = mannwhitneyu(ios_watch_category, smart_tv_watch_category, alternative='two-sided')

Гипотеза 4 (о распределении для дискретного показытеля): Показатель: watch_duration_category (категория длительности просмотра)

Нулевая гипотеза (H₀): Распределение категорий длительности просмотра для категорий контента Comedy и News одинаково.

Альтернативная гипотеза (H₁): Распределение категорий длительности просмотра для категорий контента Comedy и News различается.

In [26]:
comedy_watch_category = youtube_logs_custom.loc[youtube_logs_custom['category'] == 'Comedy', 'watch_duration_category'].value_counts().reindex(labels, fill_value=0)
news_watch_category = youtube_logs_custom.loc[youtube_logs_custom['category'] == 'News', 'watch_duration_category'].value_counts().reindex(labels, fill_value=0)
contingency_table = pd.DataFrame({
    'Comedy': comedy_watch_category,
    'News': news_watch_category
}).T
stat4, p4, _, _ = chi2_contingency(contingency_table)

In [27]:
results = pd.DataFrame({
    'Hypothesis': [
        'Median (Web vs Android)',
        'Distribution (Music vs Gaming)',
        'Median (iOS vs Smart TV)',
        'Distribution (Comedy vs News)'
    ],
    'Test Statistic': [stat1, stat2, stat3, stat4],
    'p-value': [p1, p2, p3, p4]
})

In [28]:
results

,Hypothesis,Test Statistic,p-value
0,Median (Web vs Android),2.740441e+07,0.285914
1,Distribution (Music vs Gaming),1.598699e-02,0.901389
2,Median (iOS vs Smart TV),1.300000e+01,0.200000
3,Distribution (Comedy vs News),2.916159e+00,0.404733


1. Гипотеза о медиане для непрерывного показателя (Web vs Android)
Тест Манна-Уитни
Результат: p = 0.285914, гипотеза H0 не отвергается, медианы схожи
2. Гипотеза о распределении для непрерывного показателя (Music vs Gaming)
Тест Колмогорова-Смирнова
Результат: p = 0.901389, гипотеза H0 не отвергается, распределения схожи
3. Гипотеза о медиане для дискретного показателя (iOS vs Smart TV)
Тест Манна-Уитни
Результат: p = 0.2000000, гипотеза H0 не отвергается, медианные категории схожи
4. Гипотеза о распределении для дискретного показателя (Comedy vs News)
Тест Хи-квадрат
Результат: p = 0.404733, гипотеза H0 не отвергается, распределения частот схожи

In [29]:
def bootstrap_test(data1, data2, func, num_iterations=1000, alpha=0.05):
    observed_diff = func(data1) - func(data2)
    combined = np.concatenate([data1, data2])
    bootstrap_diffs = []
    for _ in range(num_iterations):
        np.random.shuffle(combined)
        sample1 = combined[:len(data1)]
        sample2 = combined[len(data1):]
        bootstrap_diffs.append(func(sample1) - func(sample2))
    p_value = np.mean(np.abs(bootstrap_diffs) >= np.abs(observed_diff))
    return observed_diff, p_value

# 1. Бутстрап для медиан (Web vs Android)
obs_diff1, p_bootstrap1 = bootstrap_test(web_watch_duration, android_watch_duration, np.median)

# 2. Бутстрап для распределений (Music vs Gaming)
obs_diff2, p_bootstrap2 = bootstrap_test(music_watch_duration, gaming_watch_duration, np.mean)

# 3. Бутстрап для медианных категорий (iOS vs Smart TV)
ios_counts = ios_watch_category.values
smart_tv_counts = smart_tv_watch_category.values
obs_diff3, p_bootstrap3 = bootstrap_test(ios_counts, smart_tv_counts, np.median)

# 4. Бутстрап для распределений категорий (Comedy vs News)
comedy_counts = comedy_watch_category.values
news_counts = news_watch_category.values
obs_diff4, p_bootstrap4 = bootstrap_test(comedy_counts, news_counts, np.mean)

# Результаты бутстрапа
bootstrap_results = pd.DataFrame({
    'Hypothesis': [
        'Median (Web vs Android)',
        'Distribution (Music vs Gaming)',
        'Median (iOS vs Smart TV)',
        'Distribution (Comedy vs News)'
    ],
    'Observed Difference': [obs_diff1, obs_diff2, obs_diff3, obs_diff4],
    'Bootstrap p-value': [p_bootstrap1, p_bootstrap2, p_bootstrap3, p_bootstrap4]
})

In [30]:
bootstrap_results

,Hypothesis,Observed Difference,Bootstrap p-value
0,Median (Web vs Android),-0.291438,0.364
1,Distribution (Music vs Gaming),-0.400509,0.486
2,Median (iOS vs Smart TV),848.000000,0.138
3,Distribution (Comedy vs News),-6.750000,0.905


Бутстрап в целом подтверждает результаты классических тестов, кроме 1 гипотезы

Общий вывод: Бутстрап мощнее для сложных распределений и небольших выборок, а классические тесты эфиктивнее при больших выборках и когда гипотезы связаны с конкретными формами распределений.